In [2]:
import json
from owlready2 import get_ontology, Thing, DataProperty, ObjectProperty, FunctionalProperty
from unidecode import unidecode

# Charger les données JSON
with open('pokemons.json', 'r', encoding='utf-8') as file:
    pokemons_data = json.load(file)

# Créer une nouvelle ontologie
onto = get_ontology("http://example.org/onto/pokemon")

with onto:
    class Pokemon(Thing):
        pass

    class Type(Thing):
        pass
    
    class Stat(Thing):
        pass
    
    class Sensi(Thing):
        pass

    class hasType(ObjectProperty):
        domain = [Pokemon]
        range = [Type]
    
    class hasStat(ObjectProperty):
        domain = [Pokemon]
        range = [Stat]
        
    class hasSensi(ObjectProperty):
        domain = [Pokemon]
        range = [Sensi]
        
    class sensiType(ObjectProperty):
        domain = [Sensi]
        range = [Type]

    class nom(DataProperty, FunctionalProperty):
        domain = [Pokemon, Type, Stat, Sensi]
        range = [str]
    
    class valeur(DataProperty, FunctionalProperty): 
        domain = [Stat]
        range = [int]

    class valeurfloat(DataProperty, FunctionalProperty):
        domain = [Sensi]
        range = [float]

    types = {}

    for pokemon_data in pokemons_data:
        # Types du Pokémon
        types_instances = []
        for type_name in pokemon_data['types']:
            type_name = unidecode(type_name)
            if type_name not in types:
                types[type_name] = Type(nom=type_name)
            types_instances.append(types[type_name])
        
        pokemon_nom = unidecode(pokemon_data['nom']) 
        pokemon_instance = Pokemon(nom=pokemon_nom)
        
        # Association des types au Pokémon
        for type_instance in types_instances:
            pokemon_instance.hasType.append(type_instance)
        
        # Statistiques du Pokémon
        for stat_name, stat_value in pokemon_data['stats'].items():
            stat_name = unidecode(stat_name)  
            stat_instance = Stat(nom=stat_name, valeur=stat_value)
            pokemon_instance.hasStat.append(stat_instance)
            
        # Sensibilités du Pokémon
        for sensibilite_name, sensibilite_value in pokemon_data['sensibilities'].items():
            type_instance = types.get(unidecode(sensibilite_name), Type(nom=unidecode(sensibilite_name)))
            sensibilite_instance = Sensi(nom=unidecode(sensibilite_name), valeurfloat=sensibilite_value)
            sensibilite_instance.sensiType.append(type_instance)
            pokemon_instance.hasSensi.append(sensibilite_instance)

onto.save(file="pokemon_ontology.owl", format="rdfxml")

print("Ontologie créée et sauvegardée avec les données JSON.")


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Ontologie créée et sauvegardée avec les données JSON.


In [3]:
import json
from owlready2 import get_ontology, sync_reasoner, default_world

# Charger l'ontologie à partir du fichier sauvegardé
onto_path = "./pokemon_ontology.owl" 
onto = get_ontology(onto_path).load()

# Utiliser un moteur de raisonnement pour inférer de nouvelles connaissances
sync_reasoner()


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;c:\Users\basti\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/basti/AppData/Local/Temp/tmprqmed7k5
* Owlready2 * HermiT took 4.401258945465088 seconds
* Owlready * Reparenting pokemon.stat795: {pokemon.Stat} => {pokemon.Sensi, pokemon.Pokemon, pokemon.Type, pokemon.Stat}
* Owlready * Reparenting pokemon.stat796: {pokemon.Stat} => {pokemon.Sensi, pokemon.Pokemon, pokemon.Type, pokemon.Stat}
* Owlready * Reparenting pokemon.stat793: {pokemon.Stat} => {pokemon.Sensi, pokemon.Pokemon, pokemon.Type, pokemon.Stat}
* Owlready * Reparenting pokemon.stat794: {pokemon.Stat} => {pokemon.Sensi, pokemon.Pokemon, pokemon.Type, pokemon.Stat}
* Owlready * Reparenting pokemon.stat791: {pokemon.Stat} => {pokemon.Sensi, pokemon.Pokemon, pokemon.Type, pokemo

In [4]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>
SELECT DISTINCT ?pokemon WHERE {
    ?pokemon rdf:type onto:Pokemon .
    ?pokemon onto:hasType ?type .
    ?type onto:nom "Feu" .
}
"""

results = list(default_world.sparql(query))

for result in results:
    print(f"Pokémon de type Feu : {result[0].nom}")


Pokémon de type Feu : Salameche
Pokémon de type Feu : Reptincel
Pokémon de type Feu : Dracaufeu
Pokémon de type Feu : Goupix
Pokémon de type Feu : Feunard
Pokémon de type Feu : Caninos
Pokémon de type Feu : Arcanin
Pokémon de type Feu : Hericendre
Pokémon de type Feu : Feurisson
Pokémon de type Feu : Typhlosion
Pokémon de type Feu : Ponyta
Pokémon de type Feu : Galopa
Pokémon de type Feu : Ouisticram
Pokémon de type Feu : Chimpenfeu
Pokémon de type Feu : Simiabraz
Pokémon de type Feu : Poussifeu
Pokémon de type Feu : Galifeu
Pokémon de type Feu : Brasegali
Pokémon de type Feu : Magmar
Pokémon de type Feu : Limagma
Pokémon de type Feu : Volcaropod
Pokémon de type Feu : Pyroli
Pokémon de type Feu : Malosse
Pokémon de type Feu : Demolosse
Pokémon de type Feu : Sulfura
Pokémon de type Feu : Magby
Pokémon de type Feu : Entei
Pokémon de type Feu : Maganon
Pokémon de type Feu : Ho-Oh
Pokémon de type Feu : Heatran
Pokémon de type Feu : Chamallot
Pokémon de type Feu : Camerupt
Pokémon de type F

In [5]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>
SELECT DISTINCT ?otherPokemon WHERE {
    ?reptincel onto:nom "Reptincel" .
    ?reptincel onto:hasType ?reptincelType .
    ?sensi onto:sensiType ?reptincelType .
    ?sensi onto:valeurfloat ?value .
    FILTER(?value >= 4) .
    ?otherPokemon onto:hasSensi ?sensi .
}
"""

# Exécution de la requête et récupération des résultats
results = list(default_world.sparql(query))

# Affichage des résultats
for result in results:
    pokemon = result[0]
    print(f"Pokémon sensible au type de Reptincel : {pokemon.nom}")


Pokémon sensible au type de Reptincel : Paras
Pokémon sensible au type de Reptincel : Parasect
Pokémon sensible au type de Reptincel : Cheniselle
Pokémon sensible au type de Reptincel : Foretress
Pokémon sensible au type de Reptincel : Cizayox
Pokémon sensible au type de Reptincel : Blizzi
Pokémon sensible au type de Reptincel : Blizzaroi


In [6]:
def text_to_sparql(text_query):
    if "type feu" in text_query.lower():
        return """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX onto: <http://example.org/onto/pokemon#>
        SELECT DISTINCT ?pokemon WHERE {
            ?pokemon rdf:type onto:Pokemon .
            ?pokemon onto:hasType ?type .
            ?type onto:nom "Feu" .
        }
        """
    return None

query_text = "Quels sont les type feu ?"

sparql_query = text_to_sparql(query_text)

if sparql_query :    
    results = list(default_world.sparql(sparql_query))
    for result in results:
        print(f"Pokémon de type Feu : {result[0].nom}")


Pokémon de type Feu : Salameche
Pokémon de type Feu : Reptincel
Pokémon de type Feu : Dracaufeu
Pokémon de type Feu : Goupix
Pokémon de type Feu : Feunard
Pokémon de type Feu : Caninos
Pokémon de type Feu : Arcanin
Pokémon de type Feu : Hericendre
Pokémon de type Feu : Feurisson
Pokémon de type Feu : Typhlosion
Pokémon de type Feu : Ponyta
Pokémon de type Feu : Galopa
Pokémon de type Feu : Ouisticram
Pokémon de type Feu : Chimpenfeu
Pokémon de type Feu : Simiabraz
Pokémon de type Feu : Poussifeu
Pokémon de type Feu : Galifeu
Pokémon de type Feu : Brasegali
Pokémon de type Feu : Magmar
Pokémon de type Feu : Limagma
Pokémon de type Feu : Volcaropod
Pokémon de type Feu : Pyroli
Pokémon de type Feu : Malosse
Pokémon de type Feu : Demolosse
Pokémon de type Feu : Sulfura
Pokémon de type Feu : Magby
Pokémon de type Feu : Entei
Pokémon de type Feu : Maganon
Pokémon de type Feu : Ho-Oh
Pokémon de type Feu : Heatran
Pokémon de type Feu : Chamallot
Pokémon de type Feu : Camerupt
Pokémon de type F

Test joueur vs joueur

https://www.pokepedia.fr/Olga_(Conseil_4)


In [7]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>

SELECT DISTINCT ?pokemon ?typeNom WHERE {
  VALUES ?pokemonNom { "Lamantine" "Crustabri" "Cochignon" "Lokhlass" "Lippoutou" }
  ?pokemon onto:nom ?pokemonNom .
  ?pokemon onto:hasType ?type .
  ?type onto:nom ?typeNom .
}
"""

results = list(default_world.sparql(query))

for result in results:
    pokemon, typeNom = result
    print(f"{pokemon} est de type {typeNom}")


pokemon.pokemon303 est de type Glace
pokemon.pokemon303 est de type Sol
pokemon.pokemon123 est de type Eau
pokemon.pokemon123 est de type Glace
pokemon.pokemon119 est de type Eau
pokemon.pokemon119 est de type Glace
pokemon.pokemon223 est de type Glace
pokemon.pokemon223 est de type Psy
pokemon.pokemon258 est de type Eau
pokemon.pokemon258 est de type Glace


In [8]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX onto: <http://example.org/onto/pokemon#>

SELECT DISTINCT ?nomPokemonEquipe ?nomPokemonAdverse WHERE {
  VALUES ?nomPokemonAdverse { "Lamantine" "Crustabri" "Cochignon" "Lokhlass" "Lippoutou" }
  VALUES ?nomPokemonEquipe { "Dracaufeu" "Dardargnan" "Roucarnage" "Raichu" "Nosferalto" }
  
  ?pokemonAdverse onto:nom ?nomPokemonAdverse .
  ?pokemonAdverse onto:hasSensi ?sensi .
  
  ?pokemonEquipe onto:nom ?nomPokemonEquipe .
  ?pokemonEquipe onto:hasType ?typePokemonEquipe .
  
  ?sensi onto:sensiType ?typePokemonEquipe .
  ?sensi onto:valeurfloat ?value .
  
  FILTER(?value >= 2)
}
"""

# SELECT DISTINCT ?nomPokemonEquipe ?nomPokemonAdverse WHERE


# Exécution de la requête et récupération des résultats
results = list(default_world.sparql(query))

# Organiser les résultats pour chaque Pokémon de votre équipe
sensibilites = {}
for nomPokemonEquipe, nomPokemonAdverse in results:
    if nomPokemonEquipe not in sensibilites:
        sensibilites[nomPokemonEquipe] = []
    sensibilites[nomPokemonEquipe].append(nomPokemonAdverse)

# Afficher les Pokémon adverses sensibles pour chaque Pokémon de votre équipe
for pokemonEquipe, pokemonsAdverses in sensibilites.items():
    print(f"{pokemonEquipe} a un avantage contre : {', '.join(pokemonsAdverses)}")




TypeError: '<' not supported between instances of 'str' and 'int'

In [20]:
# Liste des Pokémon du joueur
pokemons_joueur = ["Dracaufeu", "Dardargnan", "Roucarnage", "Raichu", "Nosferalto"]

# Modifier la requête pour inclure seulement les Pokémon du joueur
def meilleur_pokemon_contre(adversaire_nom, pokemons_joueur):
    # Conversion de la liste des Pokémon du joueur en chaîne pour la requête SPARQL
    pokemons_joueur_filtre = ', '.join([f'"{p}"' for p in pokemons_joueur])

    
    query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX onto: <http://example.org/onto/pokemon#>
    SELECT DISTINCT ?pokemonJoueur ?typeJoueur ?sensiAdversaire ?valeurSensi
    WHERE {{
        ?pokemonJoueur onto:nom ?nomJoueur .
        FILTER (?nomJoueur IN ({pokemons_joueur_filtre})) .
        ?pokemonJoueur onto:hasType ?typeJoueur .
        ?adversaire onto:nom "{adversaire_nom}" .
        ?adversaire onto:hasSensi ?sensiAdversaire .
        ?sensiAdversaire onto:sensiType ?typeJoueur .
        ?sensiAdversaire onto:valeurfloat ?valeurSensi .
    }} ORDER BY DESC(?valeurSensi)
    """.format(pokemons_joueur_filtre=pokemons_joueur_filtre, adversaire_nom=adversaire_nom)


    # Exécuter la requête
    resultats = default_world.sparql(query)
    
    # Afficher le meilleur Pokémon
    for resultat in resultats:
        print(f"Contre {adversaire_nom}, utilisez {resultat[0].nom} avec une efficacité de {resultat[3]} grâce à son type {resultat[1].nom}.")
        break

# Exécuter la fonction pour chaque Pokémon adverse
for pokemon_adverse in ["Lamantine", "Crustabri", "Cochignon", "Lokhlass", "Lippoutou"]:
    meilleur_pokemon_contre(pokemon_adverse, pokemons_joueur)


Contre Lamantine, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Crustabri, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Cochignon, utilisez Dracaufeu avec une efficacité de 2.0 grâce à son type Feu.
Contre Lokhlass, utilisez Raichu avec une efficacité de 2.0 grâce à son type Electrik.
Contre Lippoutou, utilisez Dracaufeu avec une efficacité de 2.0 grâce à son type Feu.
